In [1]:
import platform
import os
import openai
import os
import re
import PyPDF2

In [2]:
#load api key from .env
from dotenv import load_dotenv
load_dotenv()
openai.api_key=os.getenv("OPENAI_API_KEY")

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

print('Python: ', platform.python_version())
print('re: ', re.__version__)
print('nltk: ', nltk.__version__)

Python:  3.9.0
re:  2.2.1
nltk:  3.8.1


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lnavn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Read the PDF document
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

In [5]:
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

In [6]:
text = read_pdf("ps2.pdf")
token_count = count_tokens(text)
print(f"Number of tokens: {token_count}")

Number of tokens: 30986


In [7]:
def break_up_file(tokens, chunk_size, overlap_size):
    if len(tokens) <= chunk_size:
        yield tokens
    else:
        chunk = tokens[:chunk_size]
        yield chunk
        yield from break_up_file(tokens[chunk_size-overlap_size:], chunk_size, overlap_size)

def break_up_file_to_chunks(text, chunk_size=750, overlap_size=100):
    tokens = word_tokenize(text)
    return list(break_up_file(tokens, chunk_size, overlap_size))

In [8]:
chunks = break_up_file_to_chunks(text)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} tokens")

Chunk 0: 750 tokens
Chunk 1: 750 tokens
Chunk 2: 750 tokens
Chunk 3: 750 tokens
Chunk 4: 750 tokens
Chunk 5: 750 tokens
Chunk 6: 750 tokens
Chunk 7: 750 tokens
Chunk 8: 750 tokens
Chunk 9: 750 tokens
Chunk 10: 750 tokens
Chunk 11: 750 tokens
Chunk 12: 750 tokens
Chunk 13: 750 tokens
Chunk 14: 750 tokens
Chunk 15: 750 tokens
Chunk 16: 750 tokens
Chunk 17: 750 tokens
Chunk 18: 750 tokens
Chunk 19: 750 tokens
Chunk 20: 750 tokens
Chunk 21: 750 tokens
Chunk 22: 750 tokens
Chunk 23: 750 tokens
Chunk 24: 750 tokens
Chunk 25: 750 tokens
Chunk 26: 750 tokens
Chunk 27: 750 tokens
Chunk 28: 750 tokens
Chunk 29: 750 tokens
Chunk 30: 750 tokens
Chunk 31: 750 tokens
Chunk 32: 750 tokens
Chunk 33: 750 tokens
Chunk 34: 750 tokens
Chunk 35: 750 tokens
Chunk 36: 750 tokens
Chunk 37: 750 tokens
Chunk 38: 750 tokens
Chunk 39: 750 tokens
Chunk 40: 750 tokens
Chunk 41: 750 tokens
Chunk 42: 750 tokens
Chunk 43: 750 tokens
Chunk 44: 750 tokens
Chunk 45: 750 tokens
Chunk 46: 750 tokens
Chunk 47: 436 tokens


In [9]:
def convert_to_detokenized_text(tokenized_text):
    prompt_text = " ".join(tokenized_text)
    prompt_text = prompt_text.replace(" 's", "'s")
    return prompt_text

In [10]:
prompt_response = []
chunks = break_up_file_to_chunks(text)

In [14]:
for i, chunk in enumerate(chunks):
    prompt_request = "Please generate a Table of Contents (Topic Index) for the provided document. The Table of Contents should include the topics covered in the deposition, along with the corresponding page numbers and line numbers where each topic is mentioned. Order the topics in the Table of Contents as a Topic Index.\n" + convert_to_detokenized_text(chunks[i])
    response = openai.Completion.create(
            model="text-davinci-003",
            prompt=prompt_request,
            temperature=.5,
            max_tokens=750,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
    )
    
    prompt_response.append(response["choices"][0]["text"].strip())

In [16]:
# meeting_summary = response["choices"][0]["text"].strip()
for content in prompt_response:
    print(content)

them 01:16 18 now . Otherwise , we will proceed with the deposition . 01:16 19 MR . PURCELL : No objections . 01:17 20 THE VIDEOGRAPHER : All right . Will counsel and the 01:17 21 witness please state your names for the record ? 01:17 22 MR . PURCELL : John Purcell for the Defense . 01:18 23 THE WITNESS : Persis Yu . 01:18 24 THE VIDEOGRAPHER : Thank you . 01:18 25 Page 6 Veritext Legal Solutions 866 299-51271 Sacramento , California ; Tuesday , March 28 , 2023 ; 2 1:15 P.M. 3 4 Table of Contents 

Topic Index 
Appearances ................................. 1
Deposition of Persis Yu .................... 1
Examination of Persis Yu ................... 7
Exhibits .................................... 10
299-51271

Table of Contents

I. Introduction to Deposition ................................................................. Page 1, Lines 1-18

II. Appearances and Affiliations ............................................................ Page 1, Lines 19-26

III. Examination ..............